# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [21]:
# Set up code checking
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
    os.symlink("../input/home-data-for-ml-course/sample_submission.csv", "../input/sample_submission.csv")
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex3 import *
print("Setup Complete")

In this exercise, we will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, test_size=0.2,random_state=0)

Use the next code cell to print the first five rows of the data.

In [4]:
X_train.head

Notice that the dataset contains both numerical and categorical variables.  We'll need to encode the categorical data before training a model.

To compare different models, we'll use the same `score_dataset()` function from the tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 1: Drop columns with categorical data

We'll get started with the most straightforward approach.  Use the code cell below to preprocess the data in `X_train` and `X_valid` to remove columns with categorical data.  Set the preprocessed DataFrames to `drop_X_train` and `drop_X_valid`, respectively.  

In [6]:
# Fill in the lines below: drop columns in training and validation data
drop_X_train = X_train.select_dtypes(exclude = ['object'])
drop_X_valid = X_valid.select_dtypes(exclude = ['object'])

Run the next code cell to get the MAE for this approach.

In [7]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

# Step 2: Ordinal encoding

### Part A

If you now write code to: 
- fit an ordinal encoder to the training data, and then 
- use it to transform both the training and validation data, 

you'll get an error.  Can you see why this is the case?  (_You'll need  to use the above output to answer this question._)

Run the code cell below to save the problematic columns to a Python list `bad_label_cols`.  Likewise, columns that can be safely ordinal encoded are stored in `good_label_cols`.

In [8]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

### Part B

Use the next code cell to ordinal encode the data in `X_train` and `X_valid`.  Set the preprocessed DataFrames to `label_X_train` and `label_X_valid`, respectively.  
- We have provided code below to drop the categorical columns in `bad_label_cols` from the dataset. 
- You should ordinal encode the categorical columns in `good_label_cols`.  

In [9]:
from sklearn.preprocessing import OrdinalEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply ordinal encoder 
myEncoder = OrdinalEncoder()

label_X_train[good_label_cols] = myEncoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = myEncoder.transform(X_valid[good_label_cols])


Run the next code cell to get the MAE for this approach.

In [10]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

So far, you've tried two different approaches to dealing with categorical variables.  And, you've seen that encoding categorical data yields better results than removing columns from the dataset.

Soon, you'll try one-hot encoding.  Before then, there's one additional topic we need to cover.  Begin by running the next code cell without changes.  

In [11]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])


# Step 3: Investigating cardinality

### Part A

The output above shows, for each column with categorical data, the number of unique values in the column.  For instance, the `'Street'` column in the training data has two unique values: `'Grvl'` and `'Pave'`, corresponding to a gravel road and a paved road, respectively.

We refer to the number of unique entries of a categorical variable as the **cardinality** of that categorical variable.  For instance, the `'Street'` variable has cardinality 2.

Use the output above to answer the questions below.

In [12]:
# How much cardinality in this dataset with more than 10 missing values?
high_cardinality_numcols = len([item for item in d if d[item] > 10])

### Part B

For large datasets with many rows, one-hot encoding can greatly expand the size of the dataset.  For this reason, we typically will only one-hot encode columns with relatively low cardinality.  Then, high cardinality columns can either be dropped from the dataset, or we can use ordinal encoding.

As an example, consider a dataset with 10,000 rows, and containing one categorical column with 100 unique entries.  
- If this column is replaced with the corresponding one-hot encoding, how many entries are added to the dataset?  
- If we instead replace the column with the ordinal encoding, how many entries are added?  

Use your answers to fill in the lines below.

In [13]:
# Using one-hot encoding to a huge amount of data would make a lot of data too
# TOO MUCH MEMORY USED
OH_entries_added = 1e4*1e2 - 1e4

# So, in order to save memory efficiently we must drop the data
label_entries_added = 0

Next, we'll experiment with one-hot encoding.  But, instead of encoding all of the categorical variables in the dataset, we'll only create a one-hot encoding for columns with cardinality less than 10.

Run the code cell below without changes to set `low_cardinality_cols` to a Python list containing the columns that will be one-hot encoded.  Likewise, `high_cardinality_cols` contains a list of categorical columns that will be dropped from the dataset.

In [14]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

# Step 4: One-hot encoding

Use the next code cell to one-hot encode the data in `X_train` and `X_valid`.  Set the preprocessed DataFrames to `OH_X_train` and `OH_X_valid`, respectively.  
- The full list of categorical columns in the dataset can be found in the Python list `object_cols`.
- You should only one-hot encode the categorical columns in `low_cardinality_cols`.  All other categorical columns should be dropped from the dataset. 

In [15]:
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!

oneHotEncoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
OH_cols_train = pd.DataFrame(oneHotEncoder.fit_transform(X_train[low_cardinality_cols])) # Your code here
OH_cols_valid = pd.DataFrame(oneHotEncoder.transform(X_valid[low_cardinality_cols])) # Your code here

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

temp_OH_X_train = X_train.drop(low_cardinality_cols, axis = 1)
temp_OH_X_valid = X_valid.drop(low_cardinality_cols, axis = 1)

OH_X_train = pd.concat([temp_OH_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([temp_OH_X_valid, OH_cols_valid], axis=1)

OH_X_train = OH_X_train.drop(high_cardinality_cols, axis = 1)
OH_X_valid = OH_X_valid.drop(high_cardinality_cols, axis = 1)

Run the next code cell to get the MAE for this approach.

In [16]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

In [17]:
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
model.fit(OH_X_train, y_train)

# Step 5 : Testing Data with encoded model
With encoded model we'll testing predict the test_x and see how accurate our data with one-hot, and some missing value preprocessing with simple imputer. We'll compare the predict test with sample_submission.csv

In [18]:
# One-hot encoding
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_encoder.fit(X_train[low_cardinality_cols])
X_cols_test = pd.DataFrame(OH_encoder.transform(X_test[low_cardinality_cols]))

X_cols_test.index = X_test.index

num_X_test = X_test.drop(low_cardinality_cols, axis = 1)

OH_x_test = pd.concat([X_cols_test, num_X_test], axis = 1)

OH_x_test = OH_x_test.drop(high_cardinality_cols, axis = 1)


In [19]:
# We'll using simple imputer to preprocessing our missing values
from sklearn.impute import SimpleImputer

missing_columns = OH_x_test.isnull().sum() > 10
missing_columns = list(missing_columns [missing_columns == True].index)

imputer = SimpleImputer(strategy = 'median')

final_X_test = pd.DataFrame(imputer.fit_transform(OH_x_test))
final_X_test.columns = OH_x_test.columns

final_X_test =  final_X_test.drop(missing_columns, axis = 1)

pred = model.predict(final_X_test)



In [23]:
valid_data = pd.read_csv('../input/sample_submission.csv', index_col = 'Id')
print(f"MAE from step 5 (One-Hot Encoding and Simple Imputer): {mean_absolute_error(pred, valid_data)}") 

In [ ]:
output = pd.DataFrame({
    'Id' : X_test.index,
    'SalePrice' : pred
})
output.to_csv('./submission.csv', index=False)